# Cheap Optical Flow: Is It Good?


## Quickstart

## Credits

Some portions of this notebook adapted from:
 * [Middlebury Flow code by Johannes Oswald](https://github.com/Johswald/flow-code-python/blob/master/readFlowFile.py)
 * [DeepDeform Demo Code](https://github.com/AljazBozic/DeepDeform)
 * [OpticalFlowToolkit by RUOTENG LI](https://github.com/liruoteng/OpticalFlowToolkit)
 * [OpenCV Samples](https://github.com/opencv/opencv/blob/master/samples/python/opt_flow.py)

In [1]:
# parameters
SHOW_DEMO_OUTPUT = False
DEMO_FPS = []

RUN_FULL_ANALYSIS = True
ALL_FPS = []

In [2]:
## Setup

!pip3 install pypng scikit-image
print()
print()
print()

import copy
import imageio
import IPython.display
import math
import os
import PIL.Image
import six
import sys
import tempfile


## General Notebook Utilities
    
def imshow(x):
    IPython.display.display(PIL.Image.fromarray(x))

def show_html(x):
    from IPython.core.display import display, HTML
    display(HTML(x))


## Create a random temporary directory for analysis library (for Spark-enabled full analysis mode)
old_cwd = os.getcwd()
tempdir = tempfile.TemporaryDirectory(suffix='_cheap_optical_flow_eval_analysis')
ALIB_SRC_DIR = tempdir.name
print("Putting analysis lib in %s" % ALIB_SRC_DIR)
os.chdir(ALIB_SRC_DIR)
!mkdir -p cheap_optical_flow_eval_analysis
!touch cheap_optical_flow_eval_analysis/__init__.py

%load_ext autoreload
%autoreload 2
sys.path.append(ALIB_SRC_DIR)


You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.



Putting analysis lib in /tmp/tmptk_89y2q_cheap_optical_flow_eval_analysis


In [3]:
%%writefile cheap_optical_flow_eval_analysis/ofp.py

## Data Model & Utility Code

import attr
import cv2
import imageio
import math
import os
import PIL.Image
import six

import numpy as np

from oarphpy import plotting as op_plt
img_to_data_uri = lambda x: op_plt.img_to_data_uri(x, format='png')

@attr.s(slots=True, eq=False, weakref_slot=False)
class OpticalFlowPair(object):
    """A flyweight for a pair of images with an optical flow field.
    Supports lazy-loading of large data attributes."""
    
#     slots = (
#         'dataset',
#         'id1',
#         'id2',
#         'img1',
#         'img2',
#         'flow',
#     )
    
#     def __init__(self, **kwargs):
#         for k in self.slots:
#             setattr(self, k, kwargs.get(k))
    
    dataset = attr.ib(type=str, default='')
    """To which dataset does this pair belong?"""
    
    id1 = attr.ib(type=str, default='')
    """Identifier or URI for the first image"""
    
    id2 = attr.ib(type=str, default='')
    """Identifier or URI for the second image"""
    
    img1 = attr.ib(default=None)
    """URI or numpy array for the first image (source image)"""

    img2 = attr.ib(default=None)
    """URI or numpy array for the second image (target image)"""
    
    flow = attr.ib(default=None)
    """A callable or numpy array representing optical flow from img1 -> img2"""
    
    def get_img1(self):
        if isinstance(self.img1, six.string_types):
            self.img1 = imageio.imread(self.img1)
        return self.img1
    
    def get_img2(self):
        if isinstance(self.img2, six.string_types):
            self.img2 = imageio.imread(self.img2)
        return self.img2
    
    def get_flow(self):
        if not isinstance(self.flow, (np.ndarray, np.generic)):
            self.flow = self.flow()
        return self.flow
    
    def to_html(self):
        im1 = self.get_img1()
        im2 = self.get_img2()
        flow = self.get_flow()
        fviz = draw_flow(im1, flow)
        html = """
            <table>
            
            <tr><td style="text-align:left"><b>Dataset:</b> {dataset}</td></tr>
            
            <tr><td style="text-align:left"><b>Source Image:</b> {id1}</td></tr>
            <tr><td><img src="{im1}" width="100%" /></td></tr>

            <tr><td style="text-align:left"><b>Target Image:</b> {id2}</td></tr>
            <tr><td><img src="{im2}" width="100%" /></td></tr>

            <tr><td style="text-align:left"><b>Flow</b></td></tr>
            <tr><td><img src="{fviz}" width="100%" /></td></tr>
            </table>
        """.format(
                dataset=self.dataset,
                id1=self.id1, id2=self.id2,
                im1=img_to_data_uri(im1), im2=img_to_data_uri(im2),
                fviz=img_to_data_uri(fviz))
        return html

def draw_flow(img, flow, step=8):
    """Based upon OpenCV sample: https://github.com/opencv/opencv/blob/master/samples/python/opt_flow.py"""
    h, w = img.shape[:2]
    y, x = np.mgrid[step/2:h:step, step/2:w:step].reshape(2,-1).astype(int)
    fx, fy = flow[y,x].T
    lines = np.vstack([x, y, x+fx, y+fy]).T.reshape(-1, 2, 2)
    lines = np.int32(lines + 0.5)
    vis = img.copy()
    cv2.polylines(vis, lines, 0, (0, 255, 0))
    for (x1, y1), (_x2, _y2) in lines:
        cv2.circle(vis, (x1, y1), 1, (0, 255, 0), -1)
    return vis


Writing cheap_optical_flow_eval_analysis/ofp.py


In [4]:
from cheap_optical_flow_eval_analysis.ofp import *

## Middlebury Optical Flow



In [5]:
# Please unzip `other-color-allframes.zip` and `other-gt-flow.zip` to a directory and provide the target below:
MIDD_DATA_ROOT = '/outer_root/host_mnt/Volumes/970-evo-raid0/middlebury-flow/'

# For the Middlebury Flow dataset, we only consider the real scenes
MIDD_SCENES = [
    {
        'input': 'other-data/Dimetrodon/frame10.png',
        'expected_out': 'other-data/Dimetrodon/frame11.png',
        'flow_gt': 'other-gt-flow/Dimetrodon/flow10.flo',
    },
        {
        'input': 'other-data/Hydrangea/frame10.png',
        'expected_out': 'other-data/Hydrangea/frame11.png',
        'flow_gt': 'other-gt-flow/Hydrangea/flow10.flo',
    },
        {
        'input': 'other-data/RubberWhale/frame10.png',
        'expected_out': 'other-data/RubberWhale/frame11.png',
        'flow_gt': 'other-gt-flow/RubberWhale/flow10.flo',
    },
]


In [6]:
%%writefile cheap_optical_flow_eval_analysis/midd.py

import attr

# Written as a functor to make it easier to pickle
@attr.s(slots=True, eq=False, weakref_slot=False)
class MiddReadFlow(object):
    path = attr.ib(default='')
    def __call__(self):
        import os
        import numpy as np
        # Based upon: https://github.com/Johswald/flow-code-python/blob/master/readFlowFile.py
        # compute colored image to visualize optical flow file .flo
        # Author: Johannes Oswald, Technical University Munich
        # Contact: johannes.oswald@tum.de
        # Date: 26/04/2017
        # For more information, check http://vision.middlebury.edu/flow/ 
        path = self.path
        assert os.path.exists(path) and path.endswith('.flo'), path
        f = open(path, 'rb')
        flo_number = np.fromfile(f, np.float32, count=1)[0]
        TAG_FLOAT = 202021.25
        assert flo_number == TAG_FLOAT, 'Flow number %r incorrect.' % flo_number
        w = np.fromfile(f, np.int32, count=1)
        h = np.fromfile(f, np.int32, count=1)

        #if error try: data = np.fromfile(f, np.float32, count=2*w[0]*h[0])
        data = np.fromfile(f, np.float32, count=int(2*w*h))

        # Reshape data into 3D array (columns, rows, bands)
        flow = np.resize(data, (int(h), int(w), 2))	
        f.close()

        # We found that there are some invalid (?) (i.e. very large) flows, so we're going
        # to ignore those for this experiment.
        invalid = (flow >= 1666)
        flow[invalid] = 0

        return flow


Writing cheap_optical_flow_eval_analysis/midd.py


In [7]:
from cheap_optical_flow_eval_analysis.midd import *

In [8]:
for i, scene in enumerate(MIDD_SCENES):
    p = OpticalFlowPair(
            dataset="Middlebury Optical Flow",
            id1=scene['input'],
            img1='file://' + os.path.join(MIDD_DATA_ROOT, scene['input']),
            id2=scene['expected_out'],
            img2='file://' + os.path.join(MIDD_DATA_ROOT, scene['expected_out']),
            flow=MiddReadFlow(os.path.join(MIDD_DATA_ROOT, scene['flow_gt'])))
    
    if RUN_FULL_ANALYSIS:
        ALL_FPS.append(copy.deepcopy(p))
    
    if SHOW_DEMO_OUTPUT:
        show_html(p.to_html() + "<br/><br/><br/>")
        DEMO_FPS.append(p)

## DeepDeform

In [9]:
# Please extract deepdeform_v1.7z to a directory and provide the target below:
DD_DATA_ROOT = '/outer_root/host_mnt/Volumes/970-evo-raid0/deepdeform_v1/'

DD_DEMO_SCENES = [
    {
        "input": "train/seq000/color/000000.jpg",
        "expected_out": "train/seq000/color/000200.jpg",
        "flow_gt": "train/seq000/optical_flow/blackdog_000000_000200.oflow",
    },
    
    {
        "input": "train/seq000/color/000000.jpg",
        "expected_out": "train/seq000/color/001200.jpg",
        "flow_gt": "train/seq000/optical_flow/blackdog_000000_001200.oflow",
    },
    
    {
        "input": "train/seq001/color/003400.jpg",
        "expected_out": "train/seq001/color/003600.jpg",
        "flow_gt": "train/seq001/optical_flow/lady_003400_003600.oflow",
    },
    
    {
        "input": "train/seq337/color/000050.jpg",
        "expected_out": "train/seq337/color/000350.jpg",
        "flow_gt": "train/seq337/optical_flow/adult_000050_000350.oflow",
    },
]


In [10]:
%%writefile cheap_optical_flow_eval_analysis/deepdeform.py

import attr

# Written as a functor to make it easier to pickle
@attr.s(slots=True, eq=False, weakref_slot=False)
class DDLoadFLow(object):
    path = attr.ib(default='')
    def __call__(self):
        path = self.path
        # Based upon https://github.com/AljazBozic/DeepDeform/blob/master/utils.py#L1
        import shutil
        import struct
        import os
        import numpy as np

        # Flow is stored row-wise in order [channels, height, width].
        assert os.path.isfile(path)

        flow_gt = None
        with open(path, 'rb') as fin:
            width = struct.unpack('I', fin.read(4))[0]
            height = struct.unpack('I', fin.read(4))[0]
            channels = struct.unpack('I', fin.read(4))[0]
            n_elems = height * width * channels

            flow = struct.unpack('f' * n_elems, fin.read(n_elems * 4))
            flow_gt = np.asarray(flow, dtype=np.float32).reshape([channels, height, width])

        # Match format used in this analysis
        flow_gt = np.moveaxis(flow_gt, 0, -1) # (h, w, 2)
        invalid_flow = flow_gt == -np.Inf
        flow_gt[invalid_flow] = 0.0
        return flow_gt


Writing cheap_optical_flow_eval_analysis/deepdeform.py


In [11]:
from cheap_optical_flow_eval_analysis.deepdeform import *

In [12]:
def dd_create_fp(info):
     return OpticalFlowPair(
                dataset="DeepDeform Semi-Synthetic Optical Flow",
                id1=scene['input'],
                img1='file://' + os.path.join(DD_DATA_ROOT, scene['input']),
                id2=scene['expected_out'],
                img2='file://' + os.path.join(DD_DATA_ROOT, scene['expected_out']),
                flow=DDLoadFLow(os.path.join(DD_DATA_ROOT, scene['flow_gt'])))

import json
DD_ALIGNMENTS = json.load(open(os.path.join(DD_DATA_ROOT, 'train_alignments.json')))
ALL_DD_SCENES = [
    {
        "input": ascene['source_color'],
        "expected_out": ascene['target_color'],
        "flow_gt": ascene['optical_flow'],
    }
    for ascene in DD_ALIGNMENTS
]

print("Found %s DeepDeform scenes" % len(ALL_DD_SCENES))
if SHOW_DEMO_OUTPUT:
    for scene in DD_DEMO_SCENES:
        p = dd_create_fp(scene)
        show_html(p.to_html())
        DEMO_FPS.append(p)

if RUN_FULL_ANALYSIS:
    for scene in ALL_DD_SCENES:
        p = dd_create_fp(scene)
        ALL_FPS.append(p)
        

Found 4540 DeepDeform scenes


## Kitti Scene Flow Benchmark (2015)


In [13]:
# Please unzip `data_scene_flow.zip` and `data_scene_flow_calib.zip` to a directory and provide that target below:
KITTI_SF15_DATA_ROOT = '/outer_root/host_mnt/Volumes/970-evo-raid0/kitti_sceneflow_scratch/'

# You have to ls flow_occ 
KITTI_SF15_DEMO_SCENES = [
    {
        'input': 'training/image_2/000000_10.png',
        'expected_out': 'training/image_2/000000_11.png',
        'flow_gt': 'training/flow_occ/000000_10.png',
    },
    {
        'input': 'training/image_2/000007_10.png',
        'expected_out': 'training/image_2/000007_11.png',
        'flow_gt': 'training/flow_occ/000007_10.png',
    },
    {
        'input': 'training/image_2/000023_10.png',
        'expected_out': 'training/image_2/000023_11.png',
        'flow_gt': 'training/flow_occ/000023_10.png',
    },
    {
        'input': 'training/image_2/000051_10.png',
        'expected_out': 'training/image_2/000051_11.png',
        'flow_gt': 'training/flow_occ/000051_10.png',
    },
    {
        'input': 'training/image_2/000003_10.png',
        'expected_out': 'training/image_2/000003_11.png',
        'flow_gt': 'training/flow_occ/000003_10.png',
    },
]

from oarphpy import util as oputil
KITTI_SF15_ALL_FLOW_OCC = [
    os.path.basename(p)
    for p in oputil.all_files_recursive(
        os.path.join(KITTI_SF15_DATA_ROOT, 'training/flow_occ'), pattern='*.png')
]
    
KITTI_SF15_ALL_SCENES = [
    {
        "input": 'training/image_2/%s' % fname,
        "expected_out": 'training/image_2/%s' % fname.replace('_10', '_11'),
        "flow_gt": 'training/flow_occ/%s' % fname,
    }
    for fname in KITTI_SF15_ALL_FLOW_OCC
]
print("Found %s KITTI SceneFlow 2015 scenes" % len(KITTI_SF15_ALL_SCENES))



Found 200 KITTI SceneFlow 2015 scenes


In [14]:
%%writefile cheap_optical_flow_eval_analysis/kittisf15.py

import attr

# Written as a functor to make it easier to pickle
@attr.s(slots=True, eq=False, weakref_slot=False)
class KITTISF15LoadFlowFromPng(object):
    path = attr.ib(default='')
    def __call__(self):
        path = self.path
        # Based upon https://github.com/liruoteng/OpticalFlowToolkit/blob/master/lib/flowlib.py#L559
        import png
        import numpy as np
        flow_object = png.Reader(filename=path)
        flow_direct = flow_object.asDirect()
        flow_data = list(flow_direct[2])
        w, h = flow_direct[3]['size']
        flow = np.zeros((h, w, 3), dtype=np.float64)
        for i in range(len(flow_data)):
            flow[i, :, 0] = flow_data[i][0::3]
            flow[i, :, 1] = flow_data[i][1::3]
            flow[i, :, 2] = flow_data[i][2::3]

        invalid_idx = (flow[:, :, 2] == 0)
        flow[:, :, 0:2] = (flow[:, :, 0:2] - 2 ** 15) / 64.0
        flow[invalid_idx, 0] = 0
        flow[invalid_idx, 1] = 0
        return flow[:, :, :2]


Writing cheap_optical_flow_eval_analysis/kittisf15.py


In [15]:
from cheap_optical_flow_eval_analysis.kittisf15 import *

In [16]:
def kitti_sf15_create_fp(info):
     return OpticalFlowPair(
                dataset="KITTI Scene Flow 2015",
                id1=scene['input'],
                img1='file://' + os.path.join(KITTI_SF15_DATA_ROOT, scene['input']),
                id2=scene['expected_out'],
                img2='file://' + os.path.join(KITTI_SF15_DATA_ROOT, scene['expected_out']),
                flow=KITTISF15LoadFlowFromPng(os.path.join(KITTI_SF15_DATA_ROOT, scene['flow_gt'])))

if SHOW_DEMO_OUTPUT:
    for scene in KITTI_SF15_DEMO_SCENES:
        p = kitti_sf15_create_fp(scene)
        show_html(p.to_html())
        DEMO_FPS.append(p)

if RUN_FULL_ANALYSIS:
    for scene in KITTI_SF15_ALL_SCENES:
        p = kitti_sf15_create_fp(scene)
        ALL_FPS.append(p)

## Reconstruction via Optical Flow

In [17]:
## Reconstruction via Optical Flow

def zero_flow(flow):
    return (flow[:, :, :2] == np.array([0, 0])).all(axis=-1)

def warp_flow_backwards(img, flow):
    """Given an image, apply the inverse of `flow`"""
    h, w = flow.shape[:2]
    flow = -flow
    flow[:,:,0] += np.arange(w)
    flow[:,:,1] += np.arange(h)[:,np.newaxis]
    res = cv2.remap(img, flow.astype(np.float32), None, cv2.INTER_LINEAR)
    return res
    
def warp_flow_forwards(img, flow):
    """Given an image, apply the given optical flow `flow`.  Returns not only the warped
    image, but a `mask` indicating warped pixels (i.e. there was non-zero flow *into* these pixels ).
    With some help from https://stackoverflow.com/questions/41703210/inverting-a-real-valued-index-grid/46009462#46009462
    """
    h, w = img.shape[:2]
    pts = flow.copy()
    pts[:, :, 0] += np.arange(w)
    pts[:, :, 1] += np.arange(h)[:, np.newaxis]
    exclude = zero_flow(flow)
    if exclude.all():
        # No flow anywhere!
        return img.copy(), np.zeros((h, w)).astype(np.bool)
    else:
        inpts = pts[~exclude]
    
    from scipy.interpolate import griddata
    inpts = np.reshape(inpts, [-1, 2])
    grid_y, grid_x = np.mgrid[:h, :w]
    chan_out = []
    for ch in range(img.shape[-1]):
        spts = img[:, :, ch][~exclude].reshape([-1, 1])
        mapped = griddata(inpts, spts, (grid_x, grid_y), method='linear')
        chan_out.append(mapped.astype(img.dtype))
    out = np.stack(chan_out, axis=-1)
    out = out.reshape([h, w, len(chan_out)])

    mask = np.reshape(inpts, [-1, 2])
    mask = np.rint(mask).astype(np.int)
    mask = mask[np.where((mask[:, 0] >= 0) & (mask[:, 0] < w) & (mask[:, 1] >= 0) & (mask[:, 1] < h))]
    valid_mask = np.zeros((h, w))
    valid_mask[mask[:, 1], mask[:, 0]] = 1
    
    return out, valid_mask.astype(np.bool)

# @attr.s(slots=True, eq=False, weakref_slot=False)
class FlowReconstructedImagePair(object):
    """A pair of reconstructed images using an input pair of images and optical
    flow field (i.e. an `OpticalFlowPair` instance)."""

    slots = (
        'opair',
        'img2_recon_fwd',
        'img2_recon_fwd_valid',
        'img1_recon_bkd',
        'img1_recon_bkd_valid'
    )
    
    def __init__(self, **kwargs):
        for k in self.slots:
            setattr(self, k, kwargs.get(k))
    
#     opair = attr.ib(default=OpticalFlowPair())
#     """The original `OpticalFlowPair` with the source of the data for this reconstruction result."""
    
#     img2_recon_fwd = attr.ib(default=np.array([]))
#     """A Numpy image containing the result of FORWARDS-WARPING OpticalFlowPair::img1
#     via OpticalFlowPair::flow to reconstruct OpticalFlowPair::img2"""

#     img2_recon_fwd_valid = attr.ib(default=np.array([]))
#     """A Numpy boolean mask indicating which pixels of `img2_recon_fwd` were modified via non-zero flow"""
    
#     img1_recon_bkd = attr.ib(default=np.array([]))
#     """A Numpy image containing the result of BACKWARDS-WARPING OpticalFlowPair::img2
#     via OpticalFlowPair::flow to reconstruct OpticalFlowPair::img1"""

#     img1_recon_bkd_valid = attr.ib(default=np.array([]))
#     """A Numpy boolean mask indicating which pixels of `img1_recon_bkd` were modified via non-zero flow"""
        
    @classmethod
    def create_from(cls, oflow_pair: OpticalFlowPair):
        flow = oflow_pair.get_flow()
        
        # Forward Warp
        fwarped, fvalid = warp_flow_forwards(oflow_pair.get_img1(), flow)

        # Backwards Warp
        exclude = zero_flow(flow)
        bwarped = warp_flow_backwards(oflow_pair.get_img2(), -flow[:, :, :2])
        bvalid = ~exclude
        
        return FlowReconstructedImagePair(
                opair=oflow_pair,
                img2_recon_fwd=fwarped,
                img2_recon_fwd_valid=fvalid,
                img1_recon_bkd=bwarped,
                img1_recon_bkd_valid=bvalid)
    
    def to_html(self):
        # We use pixels from the destination image in order to make the reconstruction 
        # easier to interpret; we'll fade them in intensity so that they are more
        # conspicuous.        
        FADE_UNTOUCHED_PIXELS = 0.3
        
        viz_fwd = self.img2_recon_fwd.copy().astype(np.float32)
        im2 = self.opair.get_img2()
        if (~self.img2_recon_fwd_valid).any():
            viz_fwd[~self.img2_recon_fwd_valid] = im2[~self.img2_recon_fwd_valid]
            viz_fwd[~self.img2_recon_fwd_valid] *= FADE_UNTOUCHED_PIXELS
        else:
            # viz_fwd = im2.copy() * FADE_UNTOUCHED_PIXELS
            print('no invalids forward!')
        
        viz_bkd = self.img1_recon_bkd.copy().astype(np.float32)
        im1 = self.opair.get_img1()
        if (~self.img1_recon_bkd_valid).any():
            viz_bkd[~self.img1_recon_bkd_valid] = im1[~self.img1_recon_bkd_valid]
            viz_bkd[~self.img1_recon_bkd_valid] *= FADE_UNTOUCHED_PIXELS
        else:
            # viz_bkd = im1.copy() * FADE_UNTOUCHED_PIXELS
            print('no invalids backwards!')
        
        html = """
            <table>
            
            <tr><td style="text-align:left"><b>Forwards Warped <i>(dark pixels unwarped)</i></b></td></tr>
            <tr><td><img src="{viz_fwd}" width="100%" /></td></tr>

            <tr><td style="text-align:left"><b>Backwards Warped <i>(dark pixels unwarped)</i></b></td></tr>
            <tr><td><img src="{viz_bkd}" width="100%" /></td></tr>

            </table>
        """.format(
                viz_fwd=img_to_data_uri(viz_fwd.astype(np.uint8)),
                viz_bkd=img_to_data_uri(viz_bkd.astype(np.uint8)))
        return html

        
if SHOW_DEMO_OUTPUT:
    DEMO_RECONS = []
    for p in DEMO_FPS:
        recon = FlowReconstructedImagePair.create_from(p)
        show_html(recon.to_html() + "</br></br></br>")
        DEMO_RECONS.append(recon)


## Analysis: Demo

In [18]:
# Analysis Utils

def mse(i1, i2, valid):
    return np.mean((i1[valid] - i2[valid]) ** 2)

def rmse(i1, i2, valid):
    return math.sqrt(mse(i1, i2, valid))

def psnr(i1, i2, valid):
    return 20 * math.log10(255) - 10 * math.log10(max((mse(i1, i2, valid), 1e-12)))

def ssim(i1, i2, valid):
    # Some variance out there ...
    # https://github.com/scikit-image/scikit-image/blob/master/skimage/metrics/_structural_similarity.py#L12-L232
    # https://github.com/nianticlabs/monodepth2/blob/13200ab2f29f2f10dec3aa5db29c32a23e29d376/layers.py#L218
    # https://cvnote.ddlee.cn/2019/09/12/psnr-ssim-python
    # We will just use SKImage for now ...
    from skimage.metrics import structural_similarity as ssim
    mssim, S = ssim(i1, i2, win_size=11, multichannel=True, full=True)
    return np.mean(S[valid])

def to_edge_im(img):
    return np.stack([
        cv2.Laplacian(cv2.cvtColor(img, cv2.COLOR_RGB2GRAY), cv2.CV_32F, ksize=1),
        cv2.Sobel(cv2.cvtColor(img, cv2.COLOR_RGB2GRAY), cv2.CV_32F, 1, 0, ksize=3),
        cv2.Sobel(cv2.cvtColor(img, cv2.COLOR_RGB2GRAY), cv2.CV_32F, 0, 1, ksize=3),
    ], axis=-1)

def edges_mse(i1, i2, valid):
    return mse(to_edge_im(i1), to_edge_im(i2), valid)


def oflow_coverage(valid):
    return valid.sum() / (valid.shape[0] * valid.shape[1])

def oflow_magnitude_hist(flow, valid, bins=50):
    flow_l2s = np.sqrt( flow[valid][:, 0] ** 2 + flow[valid][:, 1] ** 2 )
    bin_counts, bin_edges = np.histogram(flow_l2s, bins=bins)
    return bin_edges, bin_counts


# Analysis Data Model

class OFlowReconErrors(object):
    """Various measures of reconstruction error for a `FlowReconstructedImagePair` instance.
    Encapsulated as two dictionaries of stats for easy interop with Spark SQL."""

    RECONSTRUCTION_ERR_METRICS = {
        'SSIM': ssim,
        'MSE': mse,
        'RMSE': rmse,
        'PSNR': psnr,
        'Edges_MSE': edges_mse,
    }
    
    def __init__(self, recon_pair: FlowReconstructedImagePair):
        im2 = recon_pair.opair.get_img2()
        img2_recon_fwd = recon_pair.img2_recon_fwd
        img2_recon_fwd_valid = recon_pair.img2_recon_fwd_valid
        self.forward_stats = dict(
            (name, func(im2, img2_recon_fwd, img2_recon_fwd_valid))
            for name, func in self.RECONSTRUCTION_ERR_METRICS.items())
        
        im1 = recon_pair.opair.get_img1()
        img1_recon_fwd = recon_pair.img1_recon_bkd
        img1_recon_fwd_valid = recon_pair.img1_recon_bkd_valid
        self.backward_stats = dict(
            (name, func(im1, img1_recon_fwd, img1_recon_fwd_valid))
            for name, func in self.RECONSTRUCTION_ERR_METRICS.items())

    def to_html(self):
        stat_names = self.RECONSTRUCTION_ERR_METRICS.keys()

        rows = [
            """
            <tr>
              <td style="text-align:left"><b>{name}</b></td>
              <td style="text-align:left">{fwd:.2f}</td>
              <td style="text-align:left">{bkd:.2f}</td>
            </tr>
            """.format(name=name, fwd=self.forward_stats[name], bkd=self.backward_stats[name])
            for name in stat_names
        ]
        
        
        html = """
            <table>
              <tr>
                  <th></th> <th><b>Forwards Warp</b></th> <th><b>Backwards Warp</b></th>
              </tr>

              {table_rows}

            </table>
        """.format(table_rows="".join(rows))
        
        return html
            
# @attr.s(slots=True, eq=False, weakref_slot=False)
class OFlowStats(object):
    """Stats on the optical flow of a `OpticalFlowPair` instance"""

    slots = (
        'opair',
        'coverage',
        'magnitude_hist',
    )
    
    def __init__(self, **kwargs):
        for k in self.slots:
            setattr(self, k, kwargs.get(k))
    
#     opair = attr.ib(default=OpticalFlowPair())
#     """The original `OpticalFlowPair` with the source of the data for this reconstruction result."""
    
#     coverage = attr.ib(default=0)
#     """Fraction of the image with valid flow"""
    
#     magnitude_hist = attr.ib(default=[np.array([]), np.array([])])
#     """Histogram [bin edges, bin counts] of flow magnitudes"""
    
    @classmethod
    def create_from(cls, oflow_pair: OpticalFlowPair):
        flow = oflow_pair.get_flow()
        valid = ~zero_flow(flow)
        return OFlowStats(
                 opair=oflow_pair,
                 coverage=oflow_coverage(valid),
                 magnitude_hist=oflow_magnitude_hist(flow, valid))
                 
    def to_html(self):
        import matplotlib.pyplot as plt
        fig = plt.figure()
        bin_edges, bin_counts = self.magnitude_hist
        plt.bar(bin_edges[:-1], bin_counts)
        plt.title("Histogram of Flow Magnitudes")
        plt.xlabel('Flow Magnitude (pixels)')
        plt.ylabel('Count')

        hist_img = matplotlib_fig_to_img(fig)
        
        html = """
            <table>           
            <tr><td style="text-align:left"><b>Flow Coverage:</b> {coverage:.2f}% </td></tr>
            <tr><td><img src="{flow_hist}" width="100%" /></td></tr>
            </table>
        """.format(
                coverage=100. * self.coverage,
                flow_hist=img_to_data_uri(matplotlib_fig_to_img(hist_img)))
        return html


# Misc

def matplotlib_fig_to_img(fig):
    import io
    import matplotlib.pyplot as plt
    from PIL import Image
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    im = Image.open(buf)
    im.show()
    buf.seek(0)

    import imageio
    hist_img = imageio.imread(buf)
    buf.close()
    return hist_img


if SHOW_DEMO_OUTPUT:
    %matplotlib agg
    for recon in DEMO_RECONS:
        p = recon.opair
        errors = OFlowReconErrors(recon)
        err_html = errors.to_html()  
            
        fstats = OFlowStats.create_from(p)
        stats_html = fstats.to_html()
            
        title = "<b>{dataset} {id1} -> {id2}</b>".format(dataset=p.dataset, id1=p.id1, id2=p.id2)
        
        show_html(title + stats_html + err_html + "</br></br></br>")
            

## Analysis on Full Datasets

In [20]:
import sys
sys.path.append('/opt/psegs')

from oarphpy import plotting as pl
from oarphpy.spark import NBSpark
NBSpark.SRC_ROOT = os.path.join(ALIB_SRC_DIR, 'cheap_optical_flow_eval_analysis')
NBSpark.CONF_KV.update({
    'spark.driver.maxResultSize': '2g',
    'spark.driver.memory': '16g',
  })
spark = NBSpark.getOrCreate()


from oarphpy.spark import RowAdapter

from pyspark import Row


def flow_pair_to_full_row(fp):
    fp_lite = copy.deepcopy(fp)
    recon = FlowReconstructedImagePair.create_from(fp)
    fstats = OFlowStats.create_from(fp)
    errors = OFlowReconErrors(recon)
    
    rowdata = dict(
            fp=fp_lite,
            flow_coverage=fstats.coverage,
    )
    rowdata.update(
        ('Forwards_' + k, float(v))
        for k, v in errors.forward_stats.items())
    rowdata.update(
        ('Backwards_' + k, float(v))
        for k, v in errors.backward_stats.items())
    return RowAdapter.to_row(rowdata)
  
if RUN_FULL_ANALYSIS:
#     spark = Spark.getOrCreate()
    
#     for p in ALL_FPS:
#         import cloudpickle
#         try:
#             cloudpickle.dumps(p)
#         except Exception:
#             assert False, p
#     print('all good')
    
    import pickle
    fp_rdd = spark.sparkContext.parallelize(ALL_FPS, numSlices=200)
#     print(fp_rdd.count())
    df = spark.createDataFrame(fp_rdd.map(flow_pair_to_full_row)).persist()

    print(df.count())
    df.show(10)
    df.printSchema()

2021-02-04 21:46:17,056	oarph 9935 : Using source root /opt/oarphpy/oarphpy 
2021-02-04 21:46:17,057	oarph 9935 : Using source root /opt/oarphpy 
2021-02-04 21:46:17,060	oarph 9935 : Generating egg to /tmp/tmpq4bbhuyn_oarphpy_eggbuild ...
2021-02-04 21:46:17,094	oarph 9935 : ... done.  Egg at /tmp/tmpq4bbhuyn_oarphpy_eggbuild/oarphpy-0.0.0-py3.6.egg
2021-02-04 21:46:17,103	oarph 9935 : Using source root /tmp/tmptk_89y2q_cheap_optical_flow_eval_analysis/cheap_optical_flow_eval_analysis 
2021-02-04 21:46:17,104	oarph 9935 : Using source root /tmp/tmptk_89y2q_cheap_optical_flow_eval_analysis 
2021-02-04 21:46:17,106	oarph 9935 : Generating egg to /tmp/tmp9km09yze_oarphpy_eggbuild ...
2021-02-04 21:46:17,117	oarph 9935 : ... done.  Egg at /tmp/tmp9km09yze_oarphpy_eggbuild/cheap_optical_flow_eval_analysis-0.0.0-py3.6.egg
/opt/spark/python/pyspark/sql/session.py:366: UserWarning: Using RDD of dict to inferSchema is deprecated. Use pyspark.sql.Row instead
  warnings.warn("Using RDD of dict to

4743
+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+--------------------+--------------------+
|Backwards_Edges_MSE|     Backwards_MSE|    Backwards_PSNR|    Backwards_RMSE|    Backwards_SSIM|Forwards_Edges_MSE|      Forwards_MSE|     Forwards_PSNR|     Forwards_RMSE|     Forwards_SSIM|       flow_coverage|                  fp|
+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+--------------------+--------------------+
|   81.1972427368164| 5.826628672041516|  40.4766301925312|2.4138410618848782|0.9688232937170571|   507.72802734375| 6.529287857121507| 39.98214545073427|2.5552471225150626|0.9348639854026914|  0.9524608106199689|[cheap_optical_fl...|
|    718.86279296875|11.585101458585248| 37.49180519314

In [21]:
df.columns

['Backwards_Edges_MSE',
 'Backwards_MSE',
 'Backwards_PSNR',
 'Backwards_RMSE',
 'Backwards_SSIM',
 'Forwards_Edges_MSE',
 'Forwards_MSE',
 'Forwards_PSNR',
 'Forwards_RMSE',
 'Forwards_SSIM',
 'flow_coverage',
 'fp']

In [22]:
class Plotter(pl.HistogramWithExamplesPlotter):
    NUM_BINS = 10
    ROWS_TO_DISPLAY_PER_BUCKET = 3
    SUB_PIVOT_COL = 'fp.dataset'

    def display_bucket(self, sub_pivot, bucket_id, irows):
        import itertools
        
        row_htmls = []
        for row in itertools.islice(irows, self.ROWS_TO_DISPLAY_PER_BUCKET):
            rowdata = RowAdapter.from_row(row)
            
            fp = rowdata['fp']
            recon = FlowReconstructedImagePair.create_from(fp)
            fstats = OFlowStats.create_from(fp)
            errors = OFlowReconErrors(recon)
            
            row_html = "<br/>".join((fp.to_html(), recon.to_html(), fstats.to_html(), errors.to_html()))
            row_htmls.append(row_html)
        
        HTML = """
        <b>Pivot: {spv} Bucket: {bucket_id} </b> <br/>
        
        {row_bodies}
        """.format(
              spv=sub_pivot,
              bucket_id=bucket_id,
              row_bodies="<br/><br/><br/>".join(row_htmls))
        
        return bucket_id, HTML

plotter = Plotter()

for col in df.columns:
    col = str(col)
    if col == 'fp':
        continue
    
    fig = plotter.run(df, col)
    pl.save_bokeh_fig(fig, '/outer_root/host_mnt/Volumes/970-evo-raid0/%s.html' % col)

# from bokeh.io import output_notebook
# output_notebook()
# from bokeh.plotting import show
# show(fig)

2021-02-04 22:13:33,215	oarph 9935 : Plotting histogram for Backwards_Edges_MSE of DataFrame[Backwards_Edges_MSE: double, Backwards_MSE: double, Backwards_PSNR: double, Backwards_RMSE: double, Backwards_SSIM: double, Forwards_Edges_MSE: double, Forwards_MSE: double, Forwards_PSNR: double, Forwards_RMSE: double, Forwards_SSIM: double, flow_coverage: double, fp: struct<__pyclass__:string,dataset:string,flow:struct<__pyclass__:string,path:string>,id1:string,id2:string,img1:string,img2:string>] ...
2021-02-04 22:13:36,031	oarph 9935 : ... building data source for ALL ...
2021-02-04 22:13:36,032	oarph 9935 : ... histogramming ALL ...
2021-02-04 22:13:39,090	oarph 9935 : ... display-ifying examples for ALL ...
2021-02-04 22:14:27,230	oarph 9935 : ... building data source for DeepDeform Semi-Synthetic Optical Flow ...
2021-02-04 22:14:27,289	oarph 9935 : ... histogramming DeepDeform Semi-Synthetic Optical Flow ...
2021-02-04 22:14:31,222	oarph 9935 : ... display-ifying examples for DeepDeform

2021-02-04 22:25:39,701	oarph 9935 : ... display-ifying examples for Middlebury Optical Flow ...
2021-02-04 22:26:00,565	oarph 9935 : Wrote Bokeh figure to /outer_root/host_mnt/Volumes/970-evo-raid0/Backwards_SSIM.html
2021-02-04 22:26:00,566	oarph 9935 : Plotting histogram for Forwards_Edges_MSE of DataFrame[Backwards_Edges_MSE: double, Backwards_MSE: double, Backwards_PSNR: double, Backwards_RMSE: double, Backwards_SSIM: double, Forwards_Edges_MSE: double, Forwards_MSE: double, Forwards_PSNR: double, Forwards_RMSE: double, Forwards_SSIM: double, flow_coverage: double, fp: struct<__pyclass__:string,dataset:string,flow:struct<__pyclass__:string,path:string>,id1:string,id2:string,img1:string,img2:string>] ...
2021-02-04 22:26:02,235	oarph 9935 : ... building data source for ALL ...
2021-02-04 22:26:02,235	oarph 9935 : ... histogramming ALL ...
2021-02-04 22:26:04,948	oarph 9935 : ... display-ifying examples for ALL ...
2021-02-04 22:26:39,238	oarph 9935 : ... building data source for De

2021-02-04 22:36:23,984	oarph 9935 : ... building data source for Middlebury Optical Flow ...
2021-02-04 22:36:24,028	oarph 9935 : ... histogramming Middlebury Optical Flow ...
2021-02-04 22:36:27,330	oarph 9935 : ... display-ifying examples for Middlebury Optical Flow ...
2021-02-04 22:36:53,875	oarph 9935 : Wrote Bokeh figure to /outer_root/host_mnt/Volumes/970-evo-raid0/Forwards_SSIM.html
2021-02-04 22:36:53,877	oarph 9935 : Plotting histogram for flow_coverage of DataFrame[Backwards_Edges_MSE: double, Backwards_MSE: double, Backwards_PSNR: double, Backwards_RMSE: double, Backwards_SSIM: double, Forwards_Edges_MSE: double, Forwards_MSE: double, Forwards_PSNR: double, Forwards_RMSE: double, Forwards_SSIM: double, flow_coverage: double, fp: struct<__pyclass__:string,dataset:string,flow:struct<__pyclass__:string,path:string>,id1:string,id2:string,img1:string,img2:string>] ...
2021-02-04 22:36:55,999	oarph 9935 : ... building data source for ALL ...
2021-02-04 22:36:56,001	oarph 9935 : 